# Introduction to Interact.jl

In [ ]:
using Interact
settheme!(:bulma)

Interact.jl provides interactive widgets for IJulia. Interaction relies on [Observables.jl](https://github.com/JuliaGizmos/Observables.jl) reactive programming package. Observables provides the type `Observable` which represent time-varying values. For example, a Slider widget can be turned into a "observable of numbers". Execute the following two cells, and then move the slider. You will see that the value of `observe(s)` changes accordingly.

In [ ]:
s = slider(0:.1:1,label="Slider X:")
display(s)
display(observe(s));

Let us now inspect the types of these entities.

In [ ]:
typeof(s)

In [ ]:
display(typeof(observe(s)));
isa(observe(s), Observable)

You can have many instances of the same widget in a notebook, and they stay in sync:

In [ ]:
s

## Using Widget Signals

A slider is useless if you cannot do more with it than just watch its value. Thankfully we can transform one signal into another, which means we can transform the signal of values that the slider takes into, say a signal of it's squares:

In [ ]:
xsquared = map(x -> x*x, observe(s))

Go ahead and vary the slider to see this in action.

You can transform a signal into pretty much anything else. Let's use the Color package to produce different saturations of red:

In [ ]:
using Colors
map(x -> RGB(x, 0.5, 0.5), observe(s))

You can of course use several inputs as arguments to `map` enabling you to combine many signals. Let's create a full color-picker.

In [ ]:
r = slider(0:0.01:1, label="R")
g = slider(0:0.01:1, label="G")
b = slider(0:0.01:1, label="B")
map(display, [r,g,b]);

In [ ]:
color = map((x, y, z) -> RGB(x, y, z), observe(r), observe(g), observe(b));
color

We can use the HTML widget to write some text you can change the color of.

In [ ]:
map(color -> HTML(string("<div style='color:#", hex(color), "'>Hello, World!</div>")), color)

## The @manipulate Macro

The `@manipulate` macro lets you play with any expression using widgets. We could have, for example, used `@manipulate` to make a color picker along with our HTML output in one line of code:

In [ ]:
@manipulate for r = 0:.05:1, g = 0:.05:1, b = 0:.05:1
    HTML(string("<div style='color:#", hex(RGB(r,g,b)), "'>Color me</div>"))
end

## Signal of Widgets

You can in fact create signal of other widgets to update them reactively. We have seen one case with `HTML` above. Let us now create a signal of Sliders:

In [ ]:
x = slider(0:.1:2pi, label="x")
s = slider(-1:.05:1, value=map(a->sin(2a), observe(x)), label="sin(2x)")
c = slider(-1:.05:1, value=map(a->cos(2a), observe(x)), label="cos(2x)")
map(display, [x,s,c]);

Now vary the x slider to see sin(2x) and cos(2x) get set to their appropriate values.

But in the above case, you cannot also use sin(2x) and cos(2x) sliders as input values (without doing tricky stuff beyond the scope of this introduction). To do this more simply, we will create a separate Input signal and pass it as an argument to map, to be the signal that the widget updates. Example:

In [ ]:
fx = Observable(0.0) # A float input

In [ ]:
x = slider(0:.1:2pi, label="x")
y = slider(-1:.05:1, value=map(sin, observe(x)), label="f(x)")
connect!(observe(y), fx)
display(x)
display(y)
display(fx);

f(x) will update as x changes. But if the user slides f(x) then the `fx` signal takes the value chosen by the user.